In [1]:
import pennylane as qml 
import pennylane.numpy as pnp 
import numpy as np 
from qas import qml_gate_ops 
from qas import qml_models 
from qas import mcts
import os 
import json 
from pprint import pprint
cwd = os.getcwd() 
print(cwd)

res_file = '20211009-231603.json'
with open(os.path.join(cwd, res_file)) as f:
    res_dict = json.load(f)

print(res_dict.keys())


d:\Research\QAS-Qiskit\QAS
dict_keys(['task', 'pool', 'params', 'k', 'op_list', 'search_reward_list', 'fine_tune_loss'])


In [14]:
pool = {int(k):res_dict['pool'][k] for k in res_dict['pool'].keys()}
params = np.array(res_dict['params'])
print(params.shape)
print(res_dict['k'])
print(pool)
model = qml_models.ToffoliQMLNoiseless(params.shape[0], params.shape[1], params.shape[2], res_dict['k'], pool)
print(model.toList(params))

(15, 12, 3)
[11, 8, 11, 8, 2, 3, 0, 1, 4, 5, 1, 3, 5, 5, 1]
{0: {'Rot': [0]}, 1: {'PlaceHolder': [0]}, 2: {'Rot': [1]}, 3: {'PlaceHolder': [1]}, 4: {'Rot': [2]}, 5: {'PlaceHolder': [2]}, 6: {'CRot': [0, 1]}, 7: {'CNOT': [0, 1]}, 8: {'CRot': [1, 2]}, 9: {'CNOT': [1, 2]}, 10: {'CRot': [0, 2]}, 11: {'CNOT': [0, 2]}}
[('CNOT', [0, 2], None), ('CRot', [1, 2], [1.129680337296492, 1.5707922440908313, -0.0008633026512832973]), ('CNOT', [0, 2], None), ('CRot', [1, 2], [0.026147067452042146, -1.26553346484485, 0.8799594504776643]), ('Rot', [1], [-0.7463000265359258, 0.000728433748695468, 0.7427270846724496]), ('PlaceHolder', [1], None), ('Rot', [0], [-0.7774424770311453, 0.0006764088145414327, -0.3979643430853004]), ('PlaceHolder', [0], None), ('Rot', [2], [-0.942913291994291, -0.1571363759045717, 0.6860885220354251]), ('PlaceHolder', [2], None), ('PlaceHolder', [0], None), ('PlaceHolder', [1], None), ('PlaceHolder', [2], None), ('PlaceHolder', [2], None), ('PlaceHolder', [0], None)]


In [15]:
dev = qml.device('default.qubit', wires = 3)
@qml.qnode(dev)
def searched_circ(x):
    qml.QubitStateVector(x, wires=[0,1,2])
    qml.CNOT(wires =[0,2])
    qml.CRot(1.129680337296492, 1.5707922440908313, -0.0008633026512832973, wires=[1,2])
    qml.CNOT(wires =[0,2])
    qml.CRot(0.026147067452042146, -1.26553346484485, 0.8799594504776643, wires=[1,2])
    qml.Rot(-0.7463000265359258, 0.000728433748695468, 0.7427270846724496, wires = 1)
    qml.Rot(-0.7774424770311453, 0.0006764088145414327, -0.3979643430853004, wires = 0)
    qml.Rot(-0.942913291994291, -0.1571363759045717, 0.6860885220354251, wires = 2)
    return qml.state()

@qml.qnode(dev)
def target_circ(x):
    qml.QubitStateVector(x, wires=[0,1,2])
    qml.Toffoli(wires=[0,1,2])
    return qml.state()

computational_bases = {"0":np.array([1,0]), "1":np.array([0,1])}

In [16]:
print(qml.draw(searched_circ)(np.kron(computational_bases['1'], np.kron(computational_bases['1'], computational_bases['0']))))

 0: ──╭QubitStateVector(M0)──╭C───────────────────────────────╭C───Rot(-0.777, 0.000676, -0.398)────────────────────────────────╭┤ State 
 1: ──├QubitStateVector(M0)──│───╭C───────────────────────────│───╭C──────────────────────────────Rot(-0.746, 0.000728, 0.743)──├┤ State 
 2: ──╰QubitStateVector(M0)──╰X──╰Rot(1.13, 1.57, -0.000863)──╰X──╰Rot(0.0261, -1.27, 0.88)───────Rot(-0.943, -0.157, 0.686)────╰┤ State 
M0 =
[0 0 0 0 0 0 1 0]



In [17]:
for a in computational_bases.keys():
    for b in computational_bases.keys():
        for c in computational_bases.keys():
            state_name = a+b+c
            state = np.kron(np.kron(computational_bases[a], computational_bases[b]), computational_bases[c])
            print("State: ", state_name)
            print("State Vector: ")
            print(state)
            print("Searched Circ Output:")
            print(searched_circ(state))
            print("Target Circ Output:")
            print(target_circ(state))
            print()

State:  000
State Vector: 
[1 0 0 0 0 0 0 0]
Searched Circ Output:
[ 7.5086e-01+6.5578e-01j -1.3031e-02-7.7398e-02j  3.9920e-05+3.6089e-04j  1.5568e-05-2.3975e-05j  3.2005e-04+1.0604e-04j -1.4207e-05-2.2423e-05j  5.9748e-08+1.0728e-07j  1.7114e-09-9.5154e-09j]
Target Circ Output:
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

State:  001
State Vector: 
[0 1 0 0 0 0 0 0]
Searched Circ Output:
[7.6509e-02-1.7512e-02j 8.9281e-01+4.4354e-01j 2.4840e-05+1.4147e-05j 1.3028e-04+3.3892e-04j 2.1558e-05-1.5488e-05j 3.3649e-04+2.1265e-05j 9.5988e-09+1.1550e-09j 8.5039e-08+8.8589e-08j]
Target Circ Output:
[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

State:  010
State Vector: 
[0 0 1 0 0 0 0 0]
Searched Circ Output:
[-1.8545e-04+3.1229e-04j -2.5883e-05+8.1837e-06j  9.5495e-01-2.8724e-01j  6.7545e-02+3.1510e-02j -1.6888e-08+1.2167e-07j -6.9971e-09+5.9440e-09j  2.6008e-04-2.1472e-04j  2.5189e-05+9.7106e-07j]
Target Circ Output:
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0